In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
from glob import glob
from matplotlib import pyplot as plt
from scipy.sparse import find, csr_matrix
from smutsia.utils.viz import plot_cloud
from smutsia.point_cloud.io import load_pyntcloud
from smutsia.point_cloud.normals import get_normals
from smutsia.utils.graph import cloud_knn_graph, cloud_spherical_graph, merge_graphs
from smutsia.morphology.segmentation.metrics import z_nz_dist
from smutsia.morphology.segmentation import quasi_flat_zones

In [3]:
basedir = '../../data/semantickitti/dataset/sequences/'
sequences = sorted(os.listdir(basedir))
plyfiles = sorted(glob(os.path.join(basedir, sequences[8], 'velodyne', '*.bin')))

In [4]:
# nidx = np.random.randint(len(plyfiles))
nidx = 3721
print(nidx)
cloud = load_pyntcloud(plyfiles[nidx], add_label=True)

3721


In [6]:
plot_cloud(cloud.xyz, scalars=cloud.points.labels.values, cmap=plt.cm.Set1)

TEST FUNC


In [6]:
knn_graph = cloud_knn_graph(cloud.xyz, k = 10)
spherical_graph = cloud_spherical_graph(cloud.xyz, res_pitch=64, res_yaw=2048)
graph = merge_graphs([knn_graph, spherical_graph])

In [7]:
plot_cloud(xyz=cloud.xyz, graph=graph)

In [8]:
normals = get_normals(cloud, method='pca', k=10)

In [10]:
cloud.points['red'] = (np.abs(normals[:, 0]) * 255).astype(np.uint8)
cloud.points['green'] = (np.abs(normals[:, 1]) * 255).astype(np.uint8)
cloud.points['blue'] = (np.abs(normals[:, 2]) * 255).astype(np.uint8)
cloud.plot(notebook=False, initial_point_size=0.03)

[(-58.8148766522738, -50.17807442148316, 79.99167014006473),
 (52.59140534990769, 16.605517939923164, 15.893026901136649),
 (0.5414451481995419, -0.10227993790660489, 0.8344914414138693)]

In [11]:
src, dst, _ = find(graph)
weigths = z_nz_dist(cloud.xyz, normals, src, dst)
wg = csr_matrix((weigths, (src, dst)), shape=graph.shape)

In [12]:
cc = quasi_flat_zones(wg, threshold=0.30, debug_info=True)
plot_cloud(cloud.xyz, scalars=cc, cmap=plt.cm.Set1)

Number of connected components:  49406


In [57]:
plotter = plot_cloud(cloud.xyz)


TEST FUNC


In [44]:
plotter.picked_cells.point_arrays['vtkOriginalPointIds']


KeyError: 'vtkOriginalPointIds'

In [50]:
plotter.picked_cells is None


False

In [51]:
plotter.picked_cells

UnstructuredGrid (0x7f0ec6621a60)
  N Cells:	822
  N Points:	822
  X Bounds:	5.299e+00, 9.309e+00
  Y Bounds:	-5.298e-01, 1.187e+00
  Z Bounds:	-1.607e+00, -1.418e+00
  N Arrays:	3

In [52]:
plotter.picked_cells.point_arrays




{'vtkOriginalPointIds': array([ 68797,  68799,  68800,  68802,  68803,  68805,  68806,  68807,
         68809,  68811,  68812,  68814,  68815,  68817,  68819,  68820,
         68821,  68823,  68824,  68826,  68828,  68830,  68831,  68832,
         68833,  70902,  70903,  70904,  70906,  70908,  70909,  70911,
         70913,  70914,  70915,  70917,  70919,  70920,  70921,  70922,
         70923,  70925,  70926,  70928,  70929,  70931,  70933,  70935,
         70936,  70937,  70938,  70940,  70941,  70942,  70943,  70944,
         70946,  73031,  73032,  73034,  73035,  73037,  73039,  73040,
         73041,  73042,  73044,  73045,  73047,  73049,  73050,  73051,
         73052,  73053,  73054,  73055,  73057,  73059,  73061,  73062,
         73063,  73064,  73065,  73066,  73067,  73068,  73069,  73071,
         73072,  73074,  73075,  73077,  73078,  73079,  73080,  73082,
         73083,  73084,  73085,  73087,  73088,  73089,  73090,  73091,
         73092,  73094,  73095,  73096,  